## Bitcoin Weekly Price Prediction with Machine Learning

#### In this notebook, we will look at the historical prices of Bitcoin and forecast the closing price for next 7 days

In [ ]:
from __future__ import print_function
from __future__ import division
from __future__ import absolute_import

In [ ]:
# importing libraries

import os
import datetime # handling timestamps

import pandas as pd
import numpy as np
import tensorflow as tf

# cell outputs
import IPython
import IPython.display

# classical Time Series Tools
import statsmodels.api as api
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.arima_model import ARMA, ARIMA

# metrics
from sklearn.metrics import explained_variance_score, mean_absolute_error

# plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

mpl.rcParams['figure.figsize'] = (8,6)
mpl.rcParams['axes.grid'] = False



In [ ]:
print(tf.__version__)

In [ ]:
data_path = '../input/cryptocurrencypricehistory/coin_Bitcoin.csv'

### Loading Data

In [ ]:
bitcoin = pd.read_csv(data_path)


In [ ]:
bitcoin.head()

In [ ]:
date_time = pd.to_datetime(bitcoin.pop('Date'),format='%Y-%m-%d %H:%M:%S')

In [ ]:
# plotting the Closing price and Marketcap
plot_cols = ['Close','Marketcap']
plot_features = bitcoin[plot_cols]
plot_features.index = date_time
_ = plot_features.plot(subplots=True)

# plotting an excerpt
plot_features = bitcoin[plot_cols][:400]
plot_features.index = date_time[:400]
_ = plot_features.plot(subplots=True)


In [ ]:
# summary statistics
bitcoin.describe().transpose()

In [ ]:
# splitting the data in train, test and val

n = len(bitcoin)

train_df = bitcoin[0:int(n*0.7)]
val_df   = bitcoin[int(0.7*n):int(0.9*n)]
test_df  = bitcoin[int(0.9*n):]


In [ ]:
# normalizing the features

train_mean = train_df.mean()
train_std  = train_df.std()

train_df = (train_df-train_mean)/train_std
val_df   = (val_df-train_mean)/train_std
test_df  = (test_df-train_mean)/train_std



In [ ]:
# plotting the standard deviation for the features
df_std = (bitcoin - train_mean)/train_std
df_std = df_std[plot_cols]
df_std = df_std.melt(var_name='Column', value_name='Normalized')
ax = sns.violinplot(x='Column',y='Normalized',data=df_std)
_ = ax.set_xticklabels(df_std.keys(),rotation=90)


In [ ]:
# evaluate stationarity
def evaluate_stationarity(timeseries, timeframe=7):
    roll_mean = timeseries.rolling(window=timeframe).mean()
    roll_std  = timeseries.rolling(window=timeframe).std()
    
    # plot the rolling statistics
    orig = plt.plot(timeseries,color='blue',label='Original')
    mean = plt.plot(roll_mean,color='red',label='Rolling Mean')
    std  = plt.plot(roll_std,color='black',label='Rolling Std')
    plt.legend(loc='best')
    plt.title('Rolling Mean and Standard Deviation')
    plt.show(block=False)
    
    # perform ADF test
    df_test = adfuller(timeseries,autolag='AIC')
    df_output = pd.Series(df_test[0:4],index=['Test Statistics','p-value','#lags used','Number of Observations Used'])
    for k,v in df_test[4].items():
        df_output['Critical Value ({})'.format(k)]=v
    print(df_output)    
    


In [ ]:
train_close = train_df['Close']
train_close.index = date_time[0:int(n*0.7)]
test_close  = test_df['Close']
test_close.index = date_time[int(n*0.9):]
val_close  = val_df['Close']
val_close.index = date_time[int(n*0.7):int(n*0.9)]


evaluate_stationarity(train_close, timeframe=356)


In [ ]:
# apply some transformations to make the data stationary
logged_data = np.log(1 + train_close)
evaluate_stationarity(logged_data,timeframe=30)

In [ ]:
# decomposing the time series
from statsmodels.tsa.seasonal import seasonal_decompose
decomposition = seasonal_decompose(list(logged_data),freq=30)

In [ ]:
def plot_components(original, decomposition):
    trend = decomposition.trend
    seasonal = decomposition.seasonal
    residual = decomposition.resid 

    plt.subplot(411)
    plt.plot(original,label='Original')
    plt.legend(loc='best')
    plt.subplot(412)
    plt.plot(trend,label='Trend')
    plt.legend(loc='best')
    plt.subplot(413)
    plt.plot(seasonal,label='Seasonality')
    plt.legend(loc='best')
    plt.subplot(414)
    plt.plot(residual,label='Residual')
    plt.legend(loc='best')
    
    
plot_components(logged_data, decomposition)    

In [ ]:
logged_residual = pd.Series(decomposition.resid)
logged_residual.index = date_time[:int(0.7*n)]
logged_residual.dropna(inplace=True)
evaluate_stationarity(logged_residual,30)


In [ ]:
# to determine the best ARIMA Model, we will perform grid-search on (p,d,q)
p_range = range(1,5)
d_range = range(1,3)
q_range = range(1,5)



def search_best_arima(train,p_range,d_range,q_range):
    best_aic = np.inf
    best_model = None
    best_order = None
    
    for (p,d,q) in list(zip(p_range,d_range,q_range)):
        arima = ARIMA(train.values, order=(p,d,q)).fit(method='mle',trend='nc',disp=0)
        aic = arima.aic
        if aic < best_aic:
            best_model = arima
            best_order = (p,d,q)
            best_aic = aic
    return best_model,best_aic,best_order


arima,aic,order = search_best_arima(logged_data,p_range,d_range,q_range)

In [ ]:
print("Best AIC : ",aic)
print("Best Order : ",order)

In [ ]:
arima.plot_predict(1,3006,alpha=0.05)

### Neural Network Models


In [ ]:
required_cols = ['High','Low','Open','Close','Marketcap']

train_df = train_df[required_cols]
test_df  = test_df[required_cols]
val_df   = val_df[required_cols]

In [ ]:
class WindowGenerator():
    def __init__(self,input_width,shift,label_width,train_df=train_df,test_df=test_df,val_df=val_df,
                label_columns=None):
        # store the raw data
        self.train_df = train_df
        self.val_df   = val_df 
        self.test_df = test_df

        # work out the label column indices
        self.label_columns = label_columns 
        if label_columns is not None:
              self.label_column_indices = {name : i for i, name in enumerate(label_columns)}

        self.column_indices = {name : i for i, name in enumerate(train_df.columns)}

        # work out the window parameters
        self.input_width = input_width 
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift

        self.input_slices = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slices]

        self.label_start = self.total_window_size - self.label_width
        self.label_slice = slice(self.label_start,None)
        self.label_indices = np.arange(self.total_window_size)[self.label_slice]
        
        self._create_example_window()
        
    def split_window(self, features):
        inputs = features[:,self.input_slices,:]
        labels = features[:,self.label_slice,:]
        if self.label_columns is not None:
            labels = tf.stack(
                [labels[:,:,self.column_indices[name]] for name in self.label_columns ],
                axis=-1
            )
        inputs.set_shape([None,self.input_width,None])
        labels.set_shape([None,self.label_width,None])
        
        return inputs,labels
    
    def _create_example_window(self):
        example_window = tf.stack([
            np.array(self.train_df[:self.total_window_size]),
            np.array(self.train_df[100:100+self.total_window_size]),
            np.array(self.train_df[200:200+self.total_window_size]),
        ])
        
        self.example = self.split_window(example_window)
        
    def plot(self,model=None,plot_col='Close', max_subplots=3):
        inputs, labels = self.example
        plt.figure(figsize=(40,30))
        plot_col_index = self.column_indices[plot_col]
        max_n = min(max_subplots,len(inputs))
        for n in range(max_n):
            plt.subplot(max_n,1,n+1)
            plt.ylabel(f'{plot_col} [normed]')
            plt.plot(self.input_indices, inputs[n,:,plot_col_index],
                    label='Inputs',marker='.',zorder=-10)
            
            if self.label_columns:
                label_col_index = self.label_column_indices.get(plot_col,None)
            else:
                label_col_index = plot_col_index
            
            if label_col_index is None:
                continue
            
            plt.scatter(self.label_indices, labels[n,:,label_col_index],
                       edgecolors='k',label='Labels',c='#2ca02c',s=64)
            if model is not None:
                predictions = model(inputs)
                plt.scatter(self.label_indices,predictions[n,:,label_col_index],
                           marker='X',edgecolors='k',label='Predictions',
                           c='#ff7f0e',s=64)
            if n == 0:
                plt.legend()
            
            plt.xlabel('Time [d]')
    
    def make_dataset(self,data):
        data = np.array(data,dtype=np.float32)
        ds = tf.keras.preprocessing.timeseries_dataset_from_array(
             data=data,
             targets=None,
             sequence_length=self.total_window_size,
             sequence_stride=1,
             shuffle=True,
             batch_size=32
          
        )
        
        ds = ds.map(self.split_window)
        return ds
    
    @property
    def train(self):
        return self.make_dataset(self.train_df)
    
    @property
    def test(self):
        return self.make_dataset(self.test_df)
    
    @property
    def val(self):
        return self.make_dataset(self.val_df)
    
  
    def __repr__(self):
            return '\n'.join([
                f'Total window size: {self.total_window_size}',
                f'Input slices: {self.input_slices}',
                f'Input indices: {self.input_indices}',
                f'Label slices: {self.label_slice}',
                f'Label indices: {self.label_indices}',
                f'Label column name(s): {self.label_columns}'])


In [ ]:
window = WindowGenerator(
                   input_width=52*7,label_width=7,shift=7,label_columns=['Close'])

window.plot()

In [ ]:
window.train

In [ ]:
# LSTM Model

num_features = 1
OUT_STEPS = 7

lstm_model = tf.keras.Sequential([
    tf.keras.layers.LSTM(32,return_sequences=False),
    tf.keras.layers.Dense(OUT_STEPS*num_features),
    tf.keras.layers.Reshape([OUT_STEPS,num_features])
])


lstm_model.compile(loss=tf.losses.MeanSquaredError(),
              optimizer=tf.optimizers.Adam(),
              metrics=[tf.metrics.MeanAbsoluteError()])

history = lstm_model.fit(window.train,epochs=25,validation_data=window.val)

In [ ]:
window.plot(model=model)

In [ ]:
lstm_model.evaluate(window.val)

In [ ]:
# Conv Model
CONV_WIDTH = 90
conv_model = tf.keras.Sequential([
    # Shape [batch, time, features] -> [batch, CONV_WIDTH, features]
    tf.keras.layers.Lambda(lambda x: x[:, -CONV_WIDTH:,:]),
    # Shape => [batch, 1, conv_units]
    tf.keras.layers.Conv1D(256, activation='relu', kernel_size=(CONV_WIDTH)),
    tf.keras.layers.Dense(OUT_STEPS*num_features,
                          kernel_initializer=tf.initializers.zeros()),
    tf.keras.layers.Reshape([OUT_STEPS, num_features])                      
])

conv_model.compile(loss=tf.losses.MeanSquaredError(),
              optimizer=tf.optimizers.Adam(),
              metrics=[tf.metrics.MeanAbsoluteError()])

history = conv_model.fit(window.train,epochs=25,validation_data=window.val)

In [ ]:
window.plot(conv_model)

In [ ]:
conv_model.evaluate(window.val)